# 使用 ASE 和 Quantum Espresso 进行 DFT 计算

原始版本的 DFT教程 使用原子模拟环境（ASE）和VASP写作，本教程使用ASE和免费的 Quantum Espresso 进行，同时使用 xespresso 作为ASE和Quantum Espresso交互的界面。

## 密度泛函理论简介

### 背景知识

### 交换关联泛函

### 基组

### 赝势

### 费米温度与能带占据

### 自旋极化和磁性

## 分子体系

### 分子的定义与可视化

#### 从头创建分子

如果没有想要的分子的数据文件，或者没有从中获取它的数据库，我们需要手动定义原子几何形状。 下面是对CO分子的处理方式。 我们必须定义每个原子的类型和位置，以及原子所在的晶胞。

In [2]:
from ase import Atoms, Atom
from ase.io import write

# 创建一个Atoms类的实例
atoms = Atoms([Atom('C', [0., 0., 0.]),
               Atom('O', [1.1, 0., 0.])],
              cell=(10, 10, 10))

print('V = {0:1.0f} Angstrom^3'.format(atoms.get_volume())) # 输出所创建晶胞的体积

write('images/simple-cubic-cell.png', atoms, show_unit_cell=2) # 在images/DFT_book文件夹中创建名为simple-cubic-cell.png的图像文件

V = 1000 Angstrom^3


![co_molecule](images/simple-cubic-cell.png '一氧化碳分子')

简单立方晶胞有两个不方便的特点：

1. 由于CO分子在拐角处，其电子密度分布在盒子的8个角上，不便于后期可视化。
2. 由于立方体的几何形状，您需要相当大的立方体以确保分子的电子密度与其图像的电子密度不重叠。电子-电子相互作用是排斥的，重叠使能量显着增加。在这里，由于距离 10 Å 的周期性边界条件，CO 分子有 6 个图像。晶胞的体积是 1000 Å3.

通过将原子集中在晶胞中，第一个问题很容易解决。第二个问题可以通过使用面心立方晶格来解决，它是最密堆积的晶格。

In [3]:
from ase import Atoms, Atom
from ase.io import write

b = 7.1
atoms = Atoms([Atom('C', [0., 0., 0.]),
               Atom('O', [1.1, 0., 0.])],
              cell=[[b, b, 0.],
                    [b, 0., b],
                    [0., b, b]])

print('V = {0:1.0f} Ang^3'.format(atoms.get_volume()))

atoms.center()  # translate atoms to center of unit cell
write('images/fcc-cell.png', atoms, show_unit_cell=2)

V = 716 Ang^3


![co_in_fcc_cell](images/fcc-cell.png '放置在fcc晶胞中心的一氧化碳分子')

#### 读取结构文件

[ase.io.read](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#ase.io.read)支持多种不同的文件格式：

 Known formats:

    =========================  ===========
    format                     short name
    =========================  ===========
    GPAW restart-file          gpw
    Dacapo netCDF output file  dacapo
    Old ASE netCDF trajectory  nc
    Virtual Nano Lab file      vnl
    ASE pickle trajectory      traj
    ASE bundle trajectory      bundle
    GPAW text output           gpaw-text
    CUBE file                  cube
    XCrySDen Structure File    xsf
    Dacapo text output         dacapo-text
    XYZ-file                   xyz
    VASP POSCAR/CONTCAR file   vasp
    VASP OUTCAR file           vasp_out
    SIESTA STRUCT file         struct_out
    ABINIT input file          abinit
    V_Sim ascii file           v_sim
    Protein Data Bank          pdb
    CIF-file                   cif
    FHI-aims geometry file     aims
    FHI-aims output file       aims_out
    VTK XML Image Data         vti
    VTK XML Structured Grid    vts
    VTK XML Unstructured Grid  vtu
    TURBOMOLE coord file       tmol
    TURBOMOLE gradient file    tmol-gradient
    exciting input             exi
    AtomEye configuration      cfg
    WIEN2k structure file      struct
    DftbPlus input file        dftb
    CASTEP geom file           cell
    CASTEP output file         castep
    CASTEP trajectory file     geom
    ETSF format                etsf.nc
    DFTBPlus GEN format        gen
    CMR db/cmr-file            db
    CMR db/cmr-file            cmr
    LAMMPS dump file           lammps
    Gromacs coordinates        gro
    =========================  ===========

#### 预先定义的分子结构

[ase](https://wiki.fysik.dtu.dk/ase/index.html) 在 [ase.data.molecules](https://wiki.fysik.dtu.dk/ase/ase/data.html#molecular-data) 数据库中定义了许多分子几何结构 。例如，该数据库包括 G2/97 数据库curtiss:1063 中的分子。该数据库包含一组广泛的原子和分子，其中存在良好的实验数据，使它们可用于基准研究。

数据库中原子的坐标是使用 MP2(full)/6-31G(d) 优化的几何结构。以下为 g2 数据库中所有的分子。

In [13]:
from ase.data import g2
keys = g2.data.keys()
# print in 3 columns
for i in range(int(len(keys) / 3)):
    print('{0:25s}{1:25s}{2:25s}'.format(*tuple(list(keys)[i * 3: i * 3 + 3])))

Be                       BeH                      C                        
C2H2                     C2H4                     C2H6                     
CH                       CH2_s1A1d                CH2_s3B1d                
CH3                      CH3Cl                    CH3OH                    
CH3SH                    CH4                      CN                       
CO                       CO2                      CS                       
Cl                       Cl2                      ClF                      
ClO                      F                        F2                       
H                        H2CO                     H2O                      
H2O2                     HCN                      HCO                      
HCl                      HF                       HOCl                     
Li                       Li2                      LiF                      
LiH                      N                        N2                       
N2H4        

#### 组合 Atoms 对象

组合两个Atoms对象通常很有用，例如用于计算反应障碍或其他类型的交互。在 ase 中，我们可以简单地将两个Atoms对象相加来实现分子合并。这是在同一晶胞中获取氨和氧分子的示例。我们Atoms使用ase.Atoms.translate函数使两个分子保持3 Å的距离。

In [15]:
from ase.build import molecule
from ase.io import write

atoms1 = molecule('NH3')

atoms2 = molecule('O2')
atoms2.translate([3, 0, 0])

bothatoms = atoms1 + atoms2
bothatoms.center(5)

write('images/bothatoms.png', bothatoms, show_unit_cell=2, rotation='90x')

![bothatoms](images/bothatoms.png '组合两个分子')

### 简单性质的计算

简单性质不需要 DFT 计算。它们通常只是原子类型和几何形状的函数。

#### 获取笛卡尔坐标

如果你想要 ( x , y, z) 形式的原子坐标，使用ase.Atoms.get_positions。如果您对分数坐标感兴趣，请使用ase.Atoms.get_scaled_positions。

In [16]:
from ase.build import molecule

atoms = molecule('C6H6')  # benzene

# access properties on each atom
print(' #  sym   p_x     p_y     p_z')
print('------------------------------')
for i, atom in enumerate(atoms):
   print('{0:3d}{1:^4s}{2:-8.2f}{3:-8.2f}{4:-8.2f}'.format(i,
                                                           atom.symbol,
                                                           atom.x,
                                                           atom.y,
                                                           atom.z))

# get all properties in arrays
sym = atoms.get_chemical_symbols()
pos = atoms.get_positions()
num = atoms.get_atomic_numbers()

atom_indices = range(len(atoms))

print()
print('  # sym    at#    p_x     p_y     p_z')
print('-------------------------------------')
for i, s, n, p in zip(atom_indices, sym, num, pos):
    px, py, pz = p
    print('{0:3d}{1:>3s}{2:8d}{3:-8.2f}{4:-8.2f}{5:-8.2f}'.format(i, s, n,
                                                                  px, py, pz))

 #  sym   p_x     p_y     p_z
------------------------------
  0 C      0.00    1.40    0.00
  1 C      1.21    0.70    0.00
  2 C      1.21   -0.70    0.00
  3 C      0.00   -1.40    0.00
  4 C     -1.21   -0.70    0.00
  5 C     -1.21    0.70    0.00
  6 H      0.00    2.48    0.00
  7 H      2.15    1.24    0.00
  8 H      2.15   -1.24    0.00
  9 H      0.00   -2.48    0.00
 10 H     -2.15   -1.24    0.00
 11 H     -2.15    1.24    0.00

  # sym    at#    p_x     p_y     p_z
-------------------------------------
  0  C       6    0.00    1.40    0.00
  1  C       6    1.21    0.70    0.00
  2  C       6    1.21   -0.70    0.00
  3  C       6    0.00   -1.40    0.00
  4  C       6   -1.21   -0.70    0.00
  5  C       6   -1.21    0.70    0.00
  6  H       1    0.00    2.48    0.00
  7  H       1    2.15    1.24    0.00
  8  H       1    2.15   -1.24    0.00
  9  H       1    0.00   -2.48    0.00
 10  H       1   -2.15   -1.24    0.00
 11  H       1   -2.15    1.24    0.00


#### 分子式和分子量

我们可以使用此配方快速计算分子的分子量。我们使用ase.Atoms.get_masses获取Atoms对象中每个原子的原子质量数组，然后将它们相加。

In [18]:
from ase.build import molecule

atoms = molecule('C6H6')
masses = atoms.get_masses()

molecular_weight = masses.sum()
molecular_formula = atoms.get_chemical_formula(mode='reduce')

# note use of two lines to keep length of line reasonable
s = 'The molecular weight of {0} is {1:1.2f} gm/mol'
print(s.format(molecular_formula, molecular_weight))

The molecular weight of C6H6 is 78.11 gm/mol


注意 ase.Atoms.get_chemical_formula 的参数 `reduce=True` 收集所有的符号提供了分子式。